In [3]:
from pathlib import Path
import pandas as pd
import seaborn as sns
#reading freesurfer results
out_dir = Path("/output")
fig_dir = out_dir/"figs"
fsdata_file = out_dir/'freesurfer_out_preped.csv'
tab_data = pd.read_csv(fsdata_file, sep=',', header=0, index_col=0);

GROUPS  = ['PD','ET','NC']
n_groups = len(GROUPS);
tab_data.shape
## basic functions
from matplotlib import pyplot as plt
# distribution of large brain parts ratio
def lr_ratio(data, items_basic, items_single, items_lr):
    item_left   = [ "Left_"+x  for x in items_lr];
    item_right  = [ "Right_"+x for x in items_lr];
    items_all = items_single + item_left + item_right + items_lr;
    tmp_data = data[items_basic+items_single+item_left+item_right];
    for x in items_lr:
        tmp_data[x]  = tmp_data["Left_"+x] + tmp_data["Right_"+x]
    #for x in items_all:
    #    tmp_data[x+"_r"] = tmp_data[x]/tmp_data["eTIV"]  
    return tmp_data, items_basic+items_all

def rm_age_sex(data, var_list):
    from sklearn import linear_model
    import numpy as np
    dat = data.copy()
    nc_data = dat[dat["is_NC"] == 1]
    x_nc = np.array([np.ones(nc_data.shape[0]), np.array((nc_data["is_Male"])), np.array((nc_data["age"]))]).T; 
    x_all= np.array([np.ones(dat.shape[0]), np.array((dat["is_Male"])), np.array((dat["age"]))]).T;
    reg_list = []; new_col=[];
    for x in var_list:
        reg = linear_model.LinearRegression()
        y_nc= np.array(nc_data[x]);
        reg.fit(x_nc, y_nc);
        tmp_col = x+"_AgeSexRemoved"
        dat[tmp_col] = dat[x]-np.matmul(x_all[:,1:], reg.coef_[1:])
        dat[tmp_col+"_resid"] = dat[x]-reg.predict(x_all)
        dat[tmp_col+"_resid_per"] = (dat[x]-reg.predict(x_all))/dat[x]
        reg_list.append(reg); new_col.append(tmp_col);
    return dat, new_col, reg_list   
# plot distribution of brian tissues
x_focus = ['eTIV', 'TotalGrayVol', 'CortexVol',
       'Brain-Stem', 'SubCortGrayVol', 'CSF', 'Left-Cerebellum-Cortex',
       'Left-Cerebellum-White-Matter', 'Right-Cerebellum-Cortex',
       'Right-Cerebellum-White-Matter', 'Cerebellum-Cortex',
       'Cerebellum-White-Matter', 'CortexVol_r', 'Brain-Stem_r',
       'SubCortGrayVol_r', 'CSF_r', 'Left-Cerebellum-Cortex_r',
       'Left-Cerebellum-White-Matter_r', 'Right-Cerebellum-Cortex_r',
       'Right-Cerebellum-White-Matter_r', 'Cerebellum-Cortex_r',
       'Cerebellum-White-Matter_r'];    
#dist_plot(tmp_data, x_focus, "age-distr")

In [20]:
# QA checking: nan errors (eTIV)
tab_data[tab_data.isna().any(axis=1)]

,age,sex,diagnosis,Left_Lateral_Ventricle,Left_Inf_Lat_Vent,Left_Cerebellum_White_Matter,Left_Cerebellum_Cortex,Left_Thalamus_Proper,Left_Caudate,Left_Putamen,...,rh_S_suborbital_volume,rh_S_subparietal_volume,rh_S_temporal_inf_volume,rh_S_temporal_sup_volume,rh_S_temporal_transverse_volume,is_PD,is_ET,is_NC,is_Male,is_Female


In [15]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.special import logit, expit
import scipy.stats as stats

def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    #aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    #, 'omega_sq']
    cols = ['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)', 'eta_sq']
    aov = aov[cols]
    return aov

def check_anova(dat, feature):
    from statsmodels.formula.api import ols
    from scipy.special import logit, expit
    import scipy.stats as stats
    y_str = 'Q(\"'+feature+'\") ~ '
    yr_str = 'logit(Q(\"'+feature+'_r\")) ~ '
    yas_str = 'Q(\"'+feature+'_AgeSexRemoved\") ~ '
    yasr_str = 'logit(Q(\"'+feature+'_AgeSexRemoved_r\")) ~ '
    model1 = ols(y_str + 'C(diagnosis) + C(sex)  + age', data=dat).fit()
    model2 = ols(yr_str + 'C(diagnosis) + C(sex)  + age', data=dat).fit()
    model3 = ols(yas_str + 'C(diagnosis) + C(sex)  + age', data=dat).fit()
    model4 = ols(yasr_str + 'C(diagnosis) + C(sex)  + age', data=dat).fit()
    aov_table1 = sm.stats.anova_lm(model1, typ=2)
    aov_table2 = sm.stats.anova_lm(model2, typ=2)
    aov_table3 = sm.stats.anova_lm(model3, typ=2)
    aov_table4 = sm.stats.anova_lm(model4, typ=2)
    print(feature," Shapiro-Wilk test:", stats.shapiro(model1.resid))
    print(anova_table(aov_table1))
    print(feature+" normalized ", "Shapiro-Wilk test:", stats.shapiro(model2.resid))
    print(anova_table(aov_table2))
    print(feature+" age sex controled ", "Shapiro-Wilk test:", stats.shapiro(model3.resid))
    print(anova_table(aov_table3))
    print(feature+" age sex controled and normalized ", "Shapiro-Wilk test:", stats.shapiro(model4.resid))
    print(anova_table(aov_table4))
    return anova_table(aov_table1), anova_table(aov_table2), anova_table(aov_table3), anova_table(aov_table4)
def check_ancova(dat, feature, is_logit):
    from statsmodels.formula.api import ols
    from scipy.special import logit, expit
    import scipy.stats as stats
    if is_logit:
        y_str = 'logit(Q(\"'+feature+'_r\")) ~ '
    else:
        y_str = 'Q(\"'+feature+'\") ~ '
    model1 = ols(y_str + 'C(diagnosis) + C(sex) + age', data=dat).fit()
    model2 = ols(y_str + 'C(diagnosis) + C(sex) + age + C(diagnosis):C(sex)', data=dat).fit()
    model3 = ols(y_str + 'C(diagnosis) + C(sex) + age + C(diagnosis):age', data=dat).fit()
    model4 = ols(y_str + 'C(diagnosis) + C(sex) + age + C(sex):age', data=dat).fit()
    model5 = ols(y_str + 'C(diagnosis) + C(sex) + age + C(diagnosis):C(sex):age', data=dat).fit()
    aov_table1 = sm.stats.anova_lm(model1, typ=2)
    aov_table2 = sm.stats.anova_lm(model2, typ=2)
    aov_table3 = sm.stats.anova_lm(model3, typ=2)
    aov_table4 = sm.stats.anova_lm(model4, typ=2)
    aov_table5 = sm.stats.anova_lm(model5, typ=2)
    print(feature," Shapiro-Wilk test:", stats.shapiro(model1.resid))
    print(anova_table(aov_table1))
    print(feature+" + diagnosis:sex ", "Shapiro-Wilk test:", stats.shapiro(model2.resid))
    print(anova_table(aov_table2))
    print(feature+" + diagnosis:age ", "Shapiro-Wilk test:", stats.shapiro(model3.resid))
    print(anova_table(aov_table3))
    print(feature+" + sex:age ", "Shapiro-Wilk test:", stats.shapiro(model4.resid))
    print(anova_table(aov_table4))
    print(feature+" + diagnosis:sex:age ", "Shapiro-Wilk test:", stats.shapiro(model5.resid))
    print(anova_table(aov_table5))
    return anova_table(aov_table1), anova_table(aov_table2), anova_table(aov_table3), anova_table(aov_table4), anova_table(aov_table5)
check_ancova(rm_asr_data, "Cerebellum-Cortex", 1)

NameError: name 'y' is not defined

In [ ]:
def screen_Tukeyhsd(data, test_list):
    import statsmodels.stats.multicomp as mc
    from functools import reduce
    res_all=[]; reject_index=[];
    for i in range(len(test_list)):
        x = test_list[i];
        tmp_comp = mc.MultiComparison(data[x], data['diagnosis'])
        tmp_res = tmp_comp.tukeyhsd()
        res_all.append(tmp_res.summary())
        if sum(list(tmp_res.reject))>=2:
            reject_index.append(i)
            print(str(i)+"th Tukey HSD test positive -->> "+x)
            print(res_all[i])
            #print(res_all[i])
    return res_all, reject_index
non_feature_list = ["diagnosis", "age", "sex", "is_PD", "is_ET","is_NC", "is_Male", "is_Female"];
all_feature_list = tab_data.columns.drop(non_feature_list)
res_all, reject_index = screen_Tukeyhsd(tab_data, all_feature_list)
print(len(reject_index))
# rh_G_temp_sup-G_T_transv_volume, rh_S_temporal_transverse_thickness

In [2]:
# select the data
from sklearn import linear_model
import numpy as np
items_basic = ["diagnosis", "age", "sex", "is_PD", "is_ET","is_NC",
               "is_Male", "is_Female", "eTIV", "TotalGrayVol",];
items_single = ["CerebralWhiteMatterVol", "CortexVol", "Brain-Stem", "SubCortGrayVol", "CSF", 
                "3rd-Ventricle", "4th-Ventricle", "5th-Ventricle", "SupraTentorialVol", 
                "CC_Anterior", "CC_Central", "CC_Mid_Anterior", "CC_Mid_Posterior", "CC_Posterior"];
items_lr  = ["Inf-Lat-Vent", "Lateral-Ventricle", 
             "Cerebellum-Cortex", "Cerebellum-White-Matter", "WM-hypointensities", 
             "Accumbens-area", "Amygdala", "Hippocampus", 
             "Pallidum", "Caudate", "Putamen", "Thalamus-Proper"];
tmp_data, items_all = lr_ratio(tab_data, items_basic, items_single, items_lr);
rm_AgeSex_list = items_all[8:];
rm_as_data, rm_as_col_list, rm_as_reg_list = rm_age_sex(tmp_data, rm_AgeSex_list)
# Check regression residuals
resid_list = [x+"_resid_per" for x in rm_as_col_list ];
#rm_as_data[resid_list].plot.box(vert=False,figsize=(10,20))


/usr/local/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# test correlation between brain areas and diagnosis
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import glm
from scipy.special import logit, expit

m1_form = "Q(\"Cerebellum-Cortex\") ~ is_ET + is_PD + is_NC"
m1 = glm(formula=m1_form, data=rm_asr_data)
res1=m1.fit()
print(res1.summary2())
m2_form = "logit(Q(\"Cerebellum-Cortex_r\")) ~ is_ET + is_PD + is_NC"
m2 = glm(formula=m2_form, data=rm_asr_data)
res2=m2.fit()
print(res2.summary2())
m3_form = "Q(\"Cerebellum-Cortex_AgeSexRemoved\") ~ is_ET + is_PD + is_NC"
m3 = glm(formula=m3_form, data=rm_asr_data)
res3=m3.fit()
print(res3.summary2())
m4_form = "logit(Q(\"Cerebellum-Cortex_AgeSexRemoved_r\")) ~ is_ET + is_PD + is_NC"
m4 = glm(formula=m4_form, data=rm_asr_data)
res4=m4.fit()
print(res4.summary2())

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import glm
from scipy.special import logit, expit

m1_form = "is_ET ~ Q(\"Left-Cerebellum-Cortex_AgeSexRemoved\") + Q(\"Left-Cerebellum-White-Matter_AgeSexRemoved\") + \
Q(\"Right-Cerebellum-Cortex_AgeSexRemoved\") + Q(\"Right-Cerebellum-White-Matter_AgeSexRemoved\") "
m1 = glm(formula=m1_form, data=rm_asr_data[rm_asr_data["diagnosis"]!="PD"],)
res1=m1.fit()
print(res1.summary2())
m2_form = "is_PD ~ Q(\"Left-Cerebellum-Cortex_AgeSexRemoved\") + Q(\"Left-Cerebellum-White-Matter_AgeSexRemoved\") + \
Q(\"Right-Cerebellum-Cortex_AgeSexRemoved\") + Q(\"Right-Cerebellum-White-Matter_AgeSexRemoved\") "
m2 = glm(formula=m2_form, data=rm_asr_data[rm_asr_data["diagnosis"]!="ET"], )
res2=m2.fit()
print(res2.summary2())

In [55]:
import statsmodels.stats.multicomp as mc
comp = mc.MultiComparison(logit(rm_asr_data['Cerebellum-Cortex_AgeSexRemoved']), rm_asr_data['diagnosis'])
tbl, a1, a2 = comp.allpairtest(stats.ttest_ind, method= "bonf")
tbl

/usr/local/miniconda/lib/python3.7/site-packages/statsmodels/stats/multitest.py:150: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
/usr/local/miniconda/lib/python3.7/site-packages/statsmodels/stats/multitest.py:254: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


group1,group2,stat,pval,pval_corr,reject
ET,NC,nan,nan,nan,False
ET,PD,nan,nan,nan,False
NC,PD,nan,nan,nan,False


In [ ]:
items_focus = ['CortexVol', 'Brain-Stem', 'SubCortGrayVol', 
               'Cerebellum-Cortex','Cerebellum-White-Matter'];
t2=sns.pairplot(rm_asr_data, vars=items_focus,  hue="diagnosis", markers=["o", "s", "D"],
                diag_kind="kde", height=5)
t2.map_lower(sns.kdeplot, levels=4, color=".2")
t2.add_legend(title="brain", adjust_subtitles=True)
#t1.savefig("brain_all.jpg", figsize=(12,6.5))

In [ ]:
features_list = ['CortexVol_AgeSexRemoved', 'Brain-Stem_AgeSexRemoved', 'SubCortGrayVol_AgeSexRemoved',
                 'Cerebellum-Cortex_AgeSexRemoved','Cerebellum-White-Matter_AgeSexRemoved']
# 'Accumbens-area_AgeSexRemoved', 'Amygdala_AgeSexRemoved', 'Hippocampus_AgeSexRemoved',
    #             'Pallidum_AgeSexRemoved', 'Caudate_AgeSexRemoved', 'Putamen_AgeSexRemoved', 'Thalamus-Proper_AgeSexRemoved'
    
et_Xtrain = et_data[features_list] 
et_ytrain = et_data[['is_ET']] 
pd_Xtrain = pd_data[features_list] 
pd_ytrain = pd_data[['is_PD']] 
   
# building the model and fitting the data 
et_log_reg = sm.Logit(et_ytrain, et_Xtrain).fit() 
pd_log_reg = sm.Logit(pd_ytrain, pd_Xtrain).fit() 

print("ET reg: ", et_log_reg.summary())
print("PD reg: ", pd_log_reg.summary())

In [54]:
# one-way anova
import pandas as pd
import researchpy as rp
import scipy.stats as stats

def check_stats(data, item):
    print(rp.summary_cont(data[item]))
    print(rp.summary_cont(data[item].groupby(data['diagnosis'])))
    anova_res = stats.f_oneway(data[item][data['diagnosis'] == 'PD'],
                               data[item][data['diagnosis'] == 'ET'],
                               data[item][data['diagnosis'] == 'NC'])
    print("F=", anova_res.statistic, "p-val=", anova_res.pvalue)
    return anova_res
anova_res=check_stats(rm_asr_data, 'Cerebellum-White-Matter_AgeSexRemoved')



                                Variable      N        Mean         SD  \
0  Cerebellum-White-Matter_AgeSexRemoved  102.0  26812.9717  3188.3921   

         SE   95% Conf.    Interval  
0  315.6979  26186.7121  27439.2314  


            N        Mean         SD        SE   95% Conf.    Interval
diagnosis                                                             
ET         29  26186.6151  2990.5739  555.3356  25049.0616  27324.1686
NC         33  27145.0292  3117.8287  542.7443  26039.4952  28250.5632
PD         40  26993.1329  3392.8646  536.4590  25908.0421  28078.2236
F= 0.7992186754683067 p-val= 0.45255984569115004
